# Lab 5 - Why have biologists still not developed an HIV vaccine?
## Hidden Markov Models
Material and embedded lab.

Motivation and some exercises are variations on those available in Bioinformatics Algorithms: An Active-Learning Approach by Phillip Compeau & Pavel Pevzner.

In [23]:
debug = True # Set this to False before pushing to be graded

# Classifying the HIV Phenotype

* Today there are over 35 million people living with the disease
* 1984 - US Health and Human Services Secretary Margaret Heckler states we will have an HIV vaccine in two years
* 1997 - Bill Clinton established a new research center with the goal of developing HIV vaccine
* 2005 - Merck beins clinical trials of an HIV vaccine, but discontinued two years later because there is some evidence this actual *increased* the risk of infection
* Antiretroviral therapy is available - a drug cocktail that stabilizes the symptoms

## Vaccines

* Classical vaccines against viruses are made from surface proteins of the virus
* HIV viral envelope proteins are extremely variable
* HIV strains taken from different patients represent diverged subtypes and therefore a vaccine must be broad enough to account for this variability
* HIV has just nine genes... I mean... come on... Why can't we do this? 
* One of my favorite thing to say is biology is complicated and this is one of the hardest things for a computer scientist who is used to 1's and 0's to internalize.

## A biological problem becomes an algorithm problem
* Multiple sequence alignments tell us a lot about a protein
* HIV proteins are extreme cases where a change at a single position can affect the HIV phenotype (and thus the drug treatment)
    * We would need a different scoring matrix across different locations in the sequence which is not part of our original formulation of the biological problem
* Constructing a multiple sequence alignment from highly diverged sequences is difficult and especially difficult for our previous approaches
* We need a new formulation and a new algorithm!

## Gambling with Yakuza
Enter a simple game of heads or tails. A fair coin would have:
$$
Pr_F(H)=1/2
\hspace{0.5in}
Pr_F(T)=1/2
$$
A biased coin might have:
$$
Pr_F(H)=3/4
\hspace{0.5in}
Pr_F(T)=1/4
$$

**What is the probability that you will see a sequence that consists of 4 heads and 6 tails given that you know it's a fair coin?**

$$
Pr(\mbox{HHHHTTTTTT}|F)=(1/2)^4*(1/2)^6=(1/2)^{10}
$$

**What is the probability that you will see a sequence that consists of 4 heads and 6 tails given that you know it's a biased coin?**

$$
Pr(\mbox{HHHHTTTTTT}|F)=(3/4)^4*(1/4)^6
$$

Given a sequence of heads and tails $x=x_1x_2...x_n$ with $k$ occurances of heads:
$$
Pr(x|F)=(1/2)^n
\\
Pr(x|B)=(1/4)^{n-k}*(3/4)^k=3^k/4^n
$$

Was the dealer using a fair or biased coin?
$$
Pr(x|F)>Pr(x|B)?
$$

You can easily compute these probabilities, so let's assume the dealer switches back and forth between the fair and biased coins with $Pr(switch)=0.1$. They do this after each flip of the coin. Now how do you determine when the dealer flipped the coin, so you can maximize your winnings?

What about a sliding window? 
* What size sliding window?
* Can you think of any other problems?

## What does this have to do with biology? Consider finding CG-islands

We've spoken before about how not all nucleotides occur at the same frequency in the genome. For example, the human genome's GC-content is approximately 42%. This would mean that the probability of CG occuring in the human genome would be $0.21*0.21=4.41\%$

But it's approximately $1\%$!!! 

This is because methylation is the most common DNA modification that adds a methyl group to the cystosine nucleotide. The resulting methylated cytosine has a tendency to deaminate into thymine (i.e., mutate). This is bad... so this CG is the least common dinucleotide.

## CG-islands
Methylation is often suppresed around genes in areas called CG-islands where CG appears relatelively frequently

A great biological quesiton is where are the CG-islands in the human genome?

Well... the genome is a sequence of characters similar to a sequence of coin flip results. Within that sequence we don't know if we are in a CG-island or not in a CG-island. This IS our coin flip problem with more symbols.

<img src="https://2.bp.blogspot.com/-Lgf7NJe-MkY/TeK1zpDQ97I/AAAAAAAAADA/egoF0eM_UvQ/s1600/fairbet.png">

<img src="https://i.imgflip.com/4052le.jpg" width=400>

* $States$ - Hidden Markov Models have hidden states (e.g., $States=\{F,B\}$)
* $\pi$ - Path through the hidden states (e.g., $\pi = FFFBBFB$)
* $\Sigma$ - Emission symbols
* $Emission$ - emission probabilities (e.g., $Pr(H|F)=1/2$)
* $Transitions$ - transition probabilities from one hidden state to another (e.g., $Pr(F|B)=0.1$)

**Exercise 1** Probability of a Hidden Path Problem

Input: A hidden path $\pi$ in an HMM ($\Sigma$, $States$, $Transition$, $Emission$).

Output: The probaiblity of the path, $Pr(\pi)$.

In [24]:
def prob_path(pi,hmm):
    sigma,states,transition,emission=hmm
    prob = 1/len(states) # probability from initial state pi_0 to pi_1 (assume uniform)
    # YOUR SOLUTION HERE
    return prob

In [25]:
import pandas as pd
pi = "FFFFBBBBFBFBBB"
sigma = ["H","T"]
states = ["F","B"]
transition = pd.DataFrame([[0.9,0.1],[0.1,0.9]],index=states,columns=states)
emission = pd.DataFrame([[0.5,0.5],[0.75,0.25]],index=states,columns=sigma)
coin_hmm = (sigma,states,transition,emission) # I'm just packing these together


prob_path(pi,coin_hmm)

2.1523360500000007e-06

But what are the inputs?

In [26]:
transition

,F,B
F,0.9,0.1
B,0.1,0.9


In [27]:
emission

,H,T
F,0.50,0.50
B,0.75,0.25


## Pandas refresher
While I love Pandas, I know that viewpoint isn't always shared. Let's answer the following question again:

What if I hate Pandas so much that I want to stop using it? For the most part this is totally fine. If you get your code to work, I'm happy. To get a list:

In [28]:
#Moving from pandas to numpy to pure python
df=pd.DataFrame([[1,2],[3,4]])
arry = df.values
arry.tolist()

[[1, 2], [3, 4]]

But we are used to dealing with things a more complicated in computer science then learning a new api, so my recommendation is to get familiar with two of the main ways to access a data frame: loc and iloc.

In [29]:
df=pd.DataFrame([[1,2,3],[4,5,6]],index=["One","Two"],columns=["A","B","C"])
if debug:
    display(df.loc["One"])
    display(df.loc["Two","B"])
    display(df.iloc[0,:])
    display(df.iloc[1,2])

A    1
B    2
C    3
Name: One, dtype: int64

5

A    1
B    2
C    3
Name: One, dtype: int64

6

## Our CG-island HMM

In [30]:
# Just an example at this point
sequence = """
ACTGCTGGATGGTCACCCCCAGCACTGACTGTCTGGAAGCTCCAGGCTCAGCTCTCAGTTTCCTGGAGCA
AGTGGGAGGATGAGGACAAGGAGGAACGAGGGCACTGGCCTCCCAGGAATTGTCCCTGAGCCTCCATCCT
GCTGTCCTGAAGCTGCCCCTGAACCTCCAACTTGCTGTCCCCAGAACTGTCATTGAGCCTACACCATGCT
ATCTCTAGAGCCCGGAAAGCCCAGGGCTGGACAAACCCCACCCCTCACTCCTCCTCTGGCCCCTTCTCCC
AGCCATCAACACTTTGGAACAGCCATCAAGCCCCTTTTAATCTCTAGAAAGGTGCCTCAGTAAGGCACAG
AGAGGTCACACCAGGTGGTCATGGTGCCTTACCTGTGTCCACTGGGCCCAGGCTGGCCCTTTAAGGGTAT
GAGGGCAGAACAGCTGAGAGACCACACCCCACTTCTCAGAGAGGTCAGGGATAAAGAAAAGGGACAATGG
AAGGAAGAACTTGTGGCCAGGATGCTGAGGGTAGAGGCTGCTCCCCACAGGCACTGAGCAGAGGGGTTGA
GGGGGGAGCCTCCAGCCTCCATCCAGACAGGACCTCTGACCGCTGCTAGGGGCCCCTTCTCAGGAGGTTC
AGTCTCAGACAAGGGCTCCAGAAACTTCAGTCCATTTTCCCAAAATGGACATGATGCATCTGGCGAGTCA
GGGACCTAGAATGTCCAGGACCGAGCCTTGCTGAGGACAGAAGGGAGCAACATGCCCTGGAGGTCCTCAT
GCAGCCCAGGCCTCTGGCAGTGACCAGCCAGCACCCAGGCAACTCCTTGCTGGGTCTCCAGGCCCTGATG
GTCACAAGAAAGGCCACAGGCCAGGGCTGGGATGGGCCCAGAGCTCTGTGTGTTTCCTGGGAGGGCTGAG
"""
sequence = "".join(sequence.strip().split("\n"))

In [31]:
states = ["I","N"] # I: CG-island and N: not a CG-island
# Emission frequencies from the book
CG_island_freq = pd.DataFrame([
    [0.053,0.079,0.127,0.036],
    [0.037,0.058,0.058,0.041],
    [0.035,0.075,0.081,0.026],
    [0.024,0.105,0.115,0.05]
],index=["A","C","G","T"],columns=["A","C","G","T"])
not_CG_island_freq = pd.DataFrame([
    [0.087,0.058,0.084,0.061],
    [0.067,0.063,0.017,0.063],
    [0.053,0.053,0.063,0.042],
    [0.051,0.070,0.084,0.084]
],index=["A","C","G","T"],columns=["A","C","G","T"])
emission = []
e = []
for i in CG_island_freq.index:
    for j in CG_island_freq.columns:
        e.append(CG_island_freq.loc[i,j])
emission.append(e)
columns = []
e = []
for i in not_CG_island_freq.index:
    for j in not_CG_island_freq.columns:
        columns.append(i+j)
        e.append(not_CG_island_freq.loc[i,j])
emission.append(e)
emission = pd.DataFrame(emission,columns=columns,index=states)
emission

,AA,AC,AG,AT,CA,CC,CG,CT,GA,GC,GG,GT,TA,TC,TG,TT
I,0.053,0.079,0.127,0.036,0.037,0.058,0.058,0.041,0.035,0.075,0.081,0.026,0.024,0.105,0.115,0.050
N,0.087,0.058,0.084,0.061,0.067,0.063,0.017,0.063,0.053,0.053,0.063,0.042,0.051,0.070,0.084,0.084


In [32]:
pi = "IIIINNNNNNNNNNNNNNNNNIIII"
sigma = emission.columns
transition = pd.DataFrame([[0.0001,1-0.0001],[1-0.1,0.1],[0.05,1-0.05]],index=["start"]+states,columns=states)
transition

,I,N
start,0.0001,0.9999
I,0.9000,0.1000
N,0.0500,0.9500


In [33]:
CG_island_hmm = (sigma,states,transition,emission)

In [34]:
prob_path(pi,CG_island_hmm)

0.000584753392287407

So now we have the probability of a path through the hidden states, but how do we incorporate the sequence itself?

$$
Pr(x,\pi)=???
$$
From the chain rule you get?

$$
Pr(x,\pi)=Pr(x|\pi)*Pr(\pi)
$$
We've already implemented $Pr(\pi)$ above, so now we need to implement the probability that you emit a sequence given a known sequence of hidden states.

$$
Pr(x,\pi)=\prod_{i=1}^n emission_{\pi_i}(x_i) * transition(\pi_{i-1}\rightarrow\pi_i)
$$

**Exercise 2**: Probability of an Outcome Given a Hidden Path Problem

Given: A string $x$, followed by the alphabet $\Sigma$ from which $x$ was constructed, followed by a hidden path $\pi$, followed by the states $States$ and emission matrix $Emission$ of an HMM.

Return: The conditional probability Pr(x|π) that string x will be emitted by the HMM given the hidden path $\pi$.

In [35]:
def prob_outcome_path(x,pi,hmm):
    sigma,states,transition,emission=hmm
    prob = 1/len(states) # probability from initial state pi_0 to pi_1 (assume uniform)
    # YOUR SOLUTION HERE
    return prob

In [36]:
x = "HHHHTTHTTTHHHH"
pi = "FFFFBBBBFBFBBB"


prob_outcome_path(x,pi,coin_hmm)

8.313141825199131e-11

## Onto better questions and Dynamic Programming 
So far none of this is too interesting because we need to pass to much information to our functions and the questions our functions answer aren't very interesting. 

What *IS* interesting if is we known a sequence $x$? One thing that is interesting is finding the optimal/most likely path of hidden states. This is what would help us identify CG-islands. This is called the decoding problem, and it requires dynamic programming to implement efficiently. Luckily, we are good at that :)

**Exercise 3**: Decoding Problem

Given: A string $x$, and a defined HMM (you know... with all the same variables as before).

Return: A path that maximizes the probability Pr(x, $\pi$) over all possible paths $\pi$.

Notes: You obviously cannot just naively enumerate all possible paths and then just select the max. You must use dynamic programming.

Some things to help. Let's say that we have solved the problem up to a certain point $i$ in the sequence. We are in one of the hidden states from the set $States$. How we know what the optimal state is at that point? We keep track of the paths costs through a graph that looks like the one on the next slide. For a good example, I've found a great image, but it switches the states to whether you are happy and sad for a slightly different HMM. The concepts are exactly the same.

<img src="https://miro.medium.com/max/3200/1*JlPYICS3_t8QkKILT3fqgQ.jpeg" width=700>

In [37]:
import numpy as np
from numpy import log

def decode_path(x,hmm,debug=False,use_log=False):
    sigma,states,transition,emission=hmm
    if type(x) == str:
        x = [c for c in x] # needed for pandas stuff
    path_probs = pd.DataFrame(np.zeros((len(states),len(x))),columns=x,index=states)
    previous_states = pd.DataFrame("?",columns=x,index=states)
    for i in range(path_probs.shape[0]): # Get the number of rows
        if use_log:
            path_probs.iloc[i,0] = log(transition.loc["start",path_probs.index[i]])+log(emission.loc[path_probs.index[i],x[0]])
        else:
            path_probs.iloc[i,0] = transition.loc["start",path_probs.index[i]]*emission.loc[path_probs.index[i],x[0]]
        previous_states.iloc[i,0] = path_probs.index[i]
    if debug:
        display(path_probs)
        display(previous_states)
        
    pi = None
    # YOUR SOLUTION HERE
    return pi

In [46]:
x = "HTHTHTHTHHHHHHHTTTTTT"
sigma = ["H","T"]
states = ["F","B"]
transition = pd.DataFrame([[0.5,0.5],[0.65,0.35],[0.35,0.65]],index=["start"]+states,columns=states)
if debug:
    display(transition)
emission = pd.DataFrame([[0.5,0.5],[0.75,0.25]],index=states,columns=sigma)

coin_hmm2 = (sigma,states,transition,emission) # transition if different because I've added a start state we need

pi = decode_path(x,coin_hmm2,debug=debug)
pi

,F,B
start,0.50,0.50
F,0.65,0.35
B,0.35,0.65


,H,T,H,T,H,T,H,T,H,H,...,H,H,H,H,T,T,T,T,T,T
F,0.250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
B,0.375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,H,T,H,T,H,T,H,T,H,H,...,H,H,H,H,T,T,T,T,T,T
F,F,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
B,B,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?


,H,T,H,T,H,T,H,T,H,H,...,H,H,H,H,T,T,T,T,T,T
F,F,F,F,F,F,F,F,F,F,F,...,F,B,B,B,B,F,F,F,F,F
B,B,B,B,B,B,B,F,B,F,B,...,B,B,B,B,B,B,F,F,F,F


,H,T,H,T,H,T,H,T,H,H,...,H,H,H,H,T,T,T,T,T,T
F,0.250,0.081250,0.026406,0.008582,0.002789,0.000906,0.000295,0.000096,0.000031,0.000010,...,0.000001,5.095828e-07,2.484216e-07,1.211055e-07,5.903895e-08,1.918766e-08,6.235989e-09,2.026696e-09,6.586763e-10,2.140698e-10
B,0.375,0.060938,0.029707,0.004827,0.002353,0.000382,0.000238,0.000039,0.000025,0.000012,...,0.000003,1.419552e-06,6.920316e-07,3.373654e-07,5.482188e-08,8.908556e-09,1.678920e-09,5.456490e-10,1.773359e-10,5.763418e-11


'FFFFFFFFFBBBBBBBFFFFF'

**Exercise 4**: Likelihood problem

Given: A string $x$, followed by an HMM.

Return: The probability the $x$ was emitted by the HMM.

In [47]:
def likelihood_x(x,hmm,debug=False):
    sigma,states,transition,emission=hmm
    if type(x) == str:
        x = [c for c in x] # needed for pandas stuff
    path_probs = pd.DataFrame(np.zeros((len(states),len(x))),columns=x,index=states)
    for i in range(path_probs.shape[0]): # Get the number of rows
        path_probs.iloc[i,0] = transition.loc["start",path_probs.index[i]]*emission.loc[path_probs.index[i],x[0]]
        
    prob = None
    # YOUR SOLUTION HERE
    return prob

In [48]:
x = "HTHTHTHTHHHHHHHTTTTTT"
prob = likelihood_x(x,coin_hmm2,debug=debug)


prob

3.544483822174205e-07

## What about our CG-islands?
Let's read a snipit and the human genome.

In [49]:
import os.path

file = None
locations = ['../data/sample.fa','../csc-448-student/data/sample.fa']
for f in locations:
    if os.path.isfile(f):
        file = f
        break
print('Opening',file)
sequence = ("".join(open(file).read().upper().split("\n")[1:])).strip()[:10000]
print(sequence[:10],"...",sequence[-10:])
print(len(sequence))

Opening ../data/sample.fa
CTAACCCTAA ... GCCGACCTGG
10000


**Exercise 5 (for fun - please comment out before submitting to the autograder)**: Find the CG-islands

Given: A string $x$, followed by an HMM.

Return: A path that maximizes the probability Pr(x, $\pi$) over all possible paths $\pi$.

In [50]:
x = [sequence[i:i+2] for i in range(0,len(sequence)-1)]
pi_discard = decode_path(x,CG_island_hmm,debug=True,use_log=False)

pi = decode_path(x,CG_island_hmm,debug=True,use_log=True)

len(pi)

,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,0.000004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N,0.062994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,I,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
N,N,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?


,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,I,N,N,N,I,I,N,N,N,N,...,I,I,I,I,I,I,I,I,I,I
N,N,N,N,N,N,N,N,N,N,N,...,I,I,I,I,I,I,I,I,I,I


,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,0.000004,0.000076,0.000008,9.963934e-07,5.201174e-08,2.715013e-09,1.020628e-10,3.575683e-12,3.825758e-13,4.713153e-14,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N,0.062994,0.003052,0.000252,1.389906e-05,8.318586e-07,4.978674e-08,2.979736e-09,1.443682e-10,1.193203e-11,6.574550e-13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,-12.404524,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
N,-2.764721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,I,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?
N,N,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,?


,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,I,N,N,N,I,I,N,N,N,N,...,I,I,I,I,I,I,I,I,I,I
N,N,N,N,N,N,N,N,N,N,N,...,N,N,I,N,I,N,N,N,N,N


,CT,TA,AA,AC,CC,CC,CT,TA,AA,AC,...,AG,GC,CC,CG,GA,AC,CC,CT,TG,GG
I,-12.404524,-9.490154,-11.725139,-13.819124,-16.771796,-19.724469,-23.005433,-26.356865,-28.591850,-30.685834,...,-28231.944962,-28234.640590,-28237.593263,-28240.545935,-28244.003703,-28246.647371,-28249.600044,-28252.899588,-28255.167771,-28257.786438
N,-2.764721,-5.791943,-8.285084,-11.183690,-13.999603,-16.815517,-19.631431,-22.658654,-25.151794,-28.050400,...,-28234.086546,-28237.075303,-28239.707796,-28243.833631,-28245.785984,-28248.684589,-28251.500503,-28254.316417,-28256.844649,-28259.660563


9999

In [51]:
sections = []
for i in range(len(pi)):
    if len(sections) == 0:
        sections.append([pi[i]])
    else:
        if sections[-1][0] == pi[i]:
            sections[-1].append(pi[i])
        else:
            sections.append([pi[i]])
prettypi = " ".join(["%d%s"%(len(section),section[0]) for section in sections])
prettypi

'4034N 250I 181N 557I 4112N 165I 181N 519I'

## Profile HMMs and Training
* Profile HMMs for Sequence Alignment
   * Separate HMM for each family
   * Seeded with multiple sequence alignment
   * Trained on a lot of sequences from the same family
   * We can then ask what is the family of a sequence (with unknown family) by checking it against each HMM!
* Training
   * Learning the correct transition and emission rates are not beyond our understanding, but in the interest of time I've left them out of this online version of this class. The book has a good section on this material.

In [44]:
# Don't forget to push!